In [27]:
import pandas as pd

# Définir le chemin du fichier
chemin = r"/kaggle/input/crypto/df_export.csv"

# Importer le DataFrame
df = pd.read_csv(chemin)

# Afficher les premières lignes pour vérifier
print(df.head())


             timestamp     open     high      low    close      volume
0  2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614
1  2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818
2  2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272
3  2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577
4  2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796


In [28]:
import pandas as pd

# Définir le chemin du fichier
chemin = r"/kaggle/input/crypto/df_BTC_export.csv"

# Importer le DataFrame
df_BTC = pd.read_csv(chemin)

# Afficher les premières lignes pour vérifier
print(df_BTC.head())

             timestamp      open      high       low     close      volume
0  2021-04-01 00:00:00  58739.46  59093.00  58720.82  59061.14  359.123687
1  2021-04-01 00:05:00  59071.84  59088.95  58977.78  59030.01  167.866814
2  2021-04-01 00:10:00  59030.00  59068.05  58965.00  58965.00  117.069392
3  2021-04-01 00:15:00  58965.01  59027.09  58852.94  58883.57  178.016350
4  2021-04-01 00:20:00  58883.57  59000.00  58862.21  58946.69   90.395679


In [29]:
import pandas as pd

# Définir le chemin du fichier
chemin = r"/kaggle/input/crypto/df_BNB_export.csv"

# Importer le DataFrame
df_BNB = pd.read_csv(chemin)

# Afficher les premières lignes pour vérifier
print(df_BNB.head())

             timestamp      open      high       low     close     volume
0  2021-04-01 00:00:00  302.4591  304.1145  302.0000  303.5287  12368.489
1  2021-04-01 00:05:00  303.5287  303.5287  301.8184  301.9038  12082.175
2  2021-04-01 00:10:00  301.9044  302.3418  301.0708  301.1205   9269.677
3  2021-04-01 00:15:00  301.1415  302.3760  300.7000  301.5339   8302.305
4  2021-04-01 00:20:00  301.5486  303.2209  301.4600  302.0695   8114.147


Jointure des 3 df

In [30]:
import pandas as pd

# Assure-toi que les timestamps sont au format datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df_BTC['timestamp'] = pd.to_datetime(df_BTC['timestamp'])
df_BNB['timestamp'] = pd.to_datetime(df_BNB['timestamp'])

# Mettre timestamp comme index pour la fusion
df.set_index('timestamp', inplace=True)
df_BTC.set_index('timestamp', inplace=True)
df_BNB.set_index('timestamp', inplace=True)

# Fusionner les DataFrames avec des suffixes pour BTC et BNB
df = df.join(df_BTC, how='inner', rsuffix='_btc')  # Suffixe _btc pour BTC
df = df.join(df_BNB, how='inner', rsuffix='_bnb')  # Suffixe _bnb pour BNB

# Réinitialiser l’index pour avoir une seule colonne timestamp
df.reset_index(inplace=True)

# Vérifier le résultat
print(df.head())

            timestamp     open     high      low    close      volume  \
0 2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614   
1 2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818   
2 2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272   
3 2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577   
4 2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796   

   open_btc  high_btc   low_btc  close_btc  volume_btc  open_bnb  high_bnb  \
0  58739.46  59093.00  58720.82   59061.14  359.123687  302.4591  304.1145   
1  59071.84  59088.95  58977.78   59030.01  167.866814  303.5287  303.5287   
2  59030.00  59068.05  58965.00   58965.00  117.069392  301.9044  302.3418   
3  58965.01  59027.09  58852.94   58883.57  178.016350  301.1415  302.3760   
4  58883.57  59000.00  58862.21   58946.69   90.395679  301.5486  303.2209   

    low_bnb  close_bnb  volume_bnb  
0  302.0000   303.5287   12368.489  
1  301.8184   301.

Création d'une base test et train

In [31]:
# Convertir la colonne timestamp en datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])
# Séparer le dataset en deux : avant et après août 2024
train_start = pd.to_datetime("2021-04-01")
train_end = pd.to_datetime("2024-01-01")
test_start = pd.to_datetime("2024-01-01")
test_end = pd.to_datetime("2025-02-26")  # Jusqu'à aujourd'hui

# Séparer les données d'entraînement et de test
train_df = df[(df["timestamp"] >= train_start) & (df["timestamp"] < train_end)]
test_df = df[(df["timestamp"] >= test_start) & (df["timestamp"] <= test_end)]

In [32]:
import pandas as pd
import numpy as np

# Fonction pour calculer le RSI
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Fonction pour calculer le MACD
def compute_macd(series, short_window=12, long_window=26, signal_window=9):
    short_ema = series.ewm(span=short_window, min_periods=1).mean()
    long_ema = series.ewm(span=long_window, min_periods=1).mean()
    macd = short_ema - long_ema
    macd_signal = macd.ewm(span=signal_window, min_periods=1).mean()
    return macd, macd_signal

# Fonction pour calculer l'ATR
def compute_atr(train_df, high_col, low_col, close_col, window=14):
    high_low = train_df[high_col] - train_df[low_col]
    high_close = np.abs(train_df[high_col] - train_df[close_col].shift())
    low_close = np.abs(train_df[low_col] - train_df[close_col].shift())
    tr = pd.DataFrame({"high_low": high_low, "high_close": high_close, "low_close": low_close}).max(axis=1)
    return tr.rolling(window=window).mean()

# Fonction pour calculer les indicateurs pour ETH, BTC et BNB
def compute_indicators(train_df):
    # Indicateurs pour ETH
    train_df["rsi"] = compute_rsi(train_df["close"], window=14)
    train_df["macd"], train_df["macd_signal"] = compute_macd(train_df["close"])
    train_df["atr"] = compute_atr(train_df, "high", "low", "close")
    train_df["volatility_20d"] = train_df["close"].pct_change().rolling(20).std()
    train_df["roc_5"] = train_df["close"].pct_change(5)
    train_df["roc_10"] = train_df["close"].pct_change(10)

    # Indicateurs pour BTC
    train_df["rsi_btc"] = compute_rsi(train_df["close_btc"], window=14)
    train_df["macd_btc"], train_df["macd_signal_btc"] = compute_macd(train_df["close_btc"])
    train_df["atr_btc"] = compute_atr(train_df, "high_btc", "low_btc", "close_btc")
    train_df["volatility_20d_btc"] = train_df["close_btc"].pct_change().rolling(20).std()
    train_df["roc_5_btc"] = train_df["close_btc"].pct_change(5)
    train_df["roc_10_btc"] = train_df["close_btc"].pct_change(10)

    # Indicateurs pour BNB
    train_df["rsi_bnb"] = compute_rsi(train_df["close_bnb"], window=14)
    train_df["macd_bnb"], train_df["macd_signal_bnb"] = compute_macd(train_df["close_bnb"])
    train_df["atr_bnb"] = compute_atr(train_df, "high_bnb", "low_bnb", "close_bnb")
    train_df["volatility_20d_bnb"] = train_df["close_bnb"].pct_change().rolling(20).std()
    train_df["roc_5_bnb"] = train_df["close_bnb"].pct_change(5)
    train_df["roc_10_bnb"] = train_df["close_bnb"].pct_change(10)

    return train_df

# Exemple d’utilisation
# (Suppose que train_df est déjà fusionné avec timestamp, close, close_btc, close_bnb, etc.)
train_df = compute_indicators(train_df)
print(train_df.head())

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
<ipython-input-32-a7e5f46e889c>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["rsi"] = compute_rsi(train_df["close"], window=14)
<ipython-input-32-a7e5f46e889c>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

            timestamp     open     high      low    close      volume  \
0 2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614   
1 2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818   
2 2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272   
3 2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577   
4 2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796   

   open_btc  high_btc   low_btc  close_btc  ...  volatility_20d_btc  \
0  58739.46  59093.00  58720.82   59061.14  ...                 NaN   
1  59071.84  59088.95  58977.78   59030.01  ...                 NaN   
2  59030.00  59068.05  58965.00   58965.00  ...                 NaN   
3  58965.01  59027.09  58852.94   58883.57  ...                 NaN   
4  58883.57  59000.00  58862.21   58946.69  ...                 NaN   

   roc_5_btc  roc_10_btc  rsi_bnb  macd_bnb  macd_signal_bnb  atr_bnb  \
0        NaN         NaN      NaN  0.000000         0.000000 

<ipython-input-32-a7e5f46e889c>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["rsi_bnb"] = compute_rsi(train_df["close_bnb"], window=14)
<ipython-input-32-a7e5f46e889c>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["macd_bnb"], train_df["macd_signal_bnb"] = compute_macd(train_df["close_bnb"])
<ipython-input-32-a7e5f46e889c>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [33]:
# Appliquer les indicateurs techniques
test_df = compute_indicators(test_df)

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
<ipython-input-32-a7e5f46e889c>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["rsi"] = compute_rsi(train_df["close"], window=14)
<ipython-input-32-a7e5f46e889c>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

Recherche des paramètres optimaux de la stratégies

In [11]:
pip install deap

Note: you may need to restart the kernel to use updated packages.


In [24]:
import numpy as np
import pandas as pd
import random
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

# Backtest avec frais, délai minimum et stop loss
def backtest_strategy(df, ema_short, ema_long, rsi_threshold, macd_threshold, stop_loss, initial_capital=5000, min_trade_delay=5):
    signals = df['signal'].values
    close_prices = df['close'].values
    capital = initial_capital
    position = 0
    entry_price = 0
    last_trade_idx = -min_trade_delay - 1
    trade_count = 0
    capital_history = [initial_capital]  # Suivi du capital

    for i in range(1, len(signals)):
        if signals[i] == 1 and position == 0 and capital >= close_prices[i] * 1.0005 and (i - last_trade_idx) > min_trade_delay:
            position = 1
            entry_price = close_prices[i]
            capital -= close_prices[i] * 1.0005  # Frais d'achat 0.05%
            last_trade_idx = i
            trade_count += 1
        elif position == 1:
            if close_prices[i] < entry_price * (1 - stop_loss):  # Stop loss
                capital += close_prices[i] * 0.9995
                position = 0
                last_trade_idx = i
                trade_count += 1
            elif signals[i] == -1 and (i - last_trade_idx) > min_trade_delay:
                capital += close_prices[i] * 0.9995  # Frais de vente
                position = 0
                last_trade_idx = i
                trade_count += 1
        current_value = capital + (position * close_prices[i])
        capital_history.append(current_value)

    if position == 1:
        capital += close_prices[-1] * 0.9995
    penalty = trade_count * 5
    final_capital = capital - penalty
    df['capital'] = pd.Series(capital_history + [final_capital] * (len(df) - len(capital_history)), index=df.index)
    return final_capital

# Calcul des signaux Ichimoku sans futur
def calculate_ichimoku_signals_no_future(df):
    df = df.copy()  # Copie explicite
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)

    high_9 = df['high'].rolling(window=9, min_periods=1).max()
    low_9 = df['low'].rolling(window=9, min_periods=1).min()
    df.loc[:, 'tenkan_sen'] = (high_9 + low_9) / 2

    high_26 = df['high'].rolling(window=26, min_periods=1).max()
    low_26 = df['low'].rolling(window=26, min_periods=1).min()
    df.loc[:, 'kijun_sen'] = (high_26 + low_26) / 2

    df.loc[:, 'senkou_span_a_present'] = (df['tenkan_sen'] + df['kijun_sen']) / 2

    high_52 = df['high'].rolling(window=52, min_periods=1).max()
    low_52 = df['low'].rolling(window=52, min_periods=1).min()
    df.loc[:, 'senkou_span_b_present'] = (high_52 + low_52) / 2

    score_signal = np.zeros(len(df))
    score_signal += np.where(df['tenkan_sen'] > df['kijun_sen'], 1, -1)
    kumo_top = df[['senkou_span_a_present', 'senkou_span_b_present']].max(axis=1)
    kumo_bottom = df[['senkou_span_a_present', 'senkou_span_b_present']].min(axis=1)
    score_signal += np.where(df['close'] > kumo_top, 1, np.where(df['close'] < kumo_bottom, -1, 0))
    df.loc[:, 'ichimoku_signal'] = np.where(score_signal > 0, 1, -1)
    return df

# Sous-AG pour optimiser les EMA
def optimize_ema(train_df):
    if "FitnessMax" not in creator.__dict__:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMax)

    def create_ema_individual():
        return [random.randint(20, 100), random.randint(100, 300)]

    def evaluate_ema(individual, df):
        ema_short, ema_long = individual
        if ema_short < 1 or ema_long < 1 or ema_short >= ema_long:
            return 0,
        df_copy = df.copy()
        df_copy['ema_short'] = df_copy['close'].ewm(span=ema_short, adjust=False).mean()
        df_copy['ema_long'] = df_copy['close'].ewm(span=ema_long, adjust=False).mean()
        df_copy['signal'] = np.where(df_copy['ema_short'] > df_copy['ema_long'], 1, -1)
        return backtest_strategy(df_copy, ema_short, ema_long, 0, 0, 0.02),  # Stop loss par défaut

    ema_toolbox = base.Toolbox()
    ema_toolbox.register("individual", tools.initIterate, creator.Individual, create_ema_individual)
    ema_toolbox.register("population", tools.initRepeat, list, ema_toolbox.individual)
    ema_toolbox.register("mate", tools.cxTwoPoint)
    ema_toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
    ema_toolbox.register("select", tools.selTournament, tournsize=3)
    ema_toolbox.register("evaluate", evaluate_ema, df=train_df)

    population = ema_toolbox.population(n=10)
    for gen in range(20):
        offspring = ema_toolbox.select(population, len(population))
        offspring = list(map(ema_toolbox.clone, offspring))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:
                ema_toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.2:
                ema_toolbox.mutate(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(ema_toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        population[:] = offspring

    best_ema = tools.selBest(population, 1)[0]
    return best_ema[0], best_ema[1]

# Générer les signaux
def generate_signals(df, ema_short, ema_long, rsi_threshold, macd_threshold):
    df = calculate_ichimoku_signals_no_future(df)
    df['ema_short'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    df['ema_long'] = df['close'].ewm(span=ema_long, adjust=False).mean()
    df['eth_btc_ratio'] = df['close'] / df['close_btc']
    df['ratio_sma'] = df['eth_btc_ratio'].rolling(window=20, min_periods=1).mean()
    df['ratio_std'] = df['eth_btc_ratio'].rolling(window=20, min_periods=1).std()
    df['upper_band'] = df['ratio_sma'] + 2 * df['ratio_std']
    df['lower_band'] = df['ratio_sma'] - 2 * df['ratio_std']

    score_signal = np.zeros(len(df))
    score_signal += np.where(df['ema_short'] > df['ema_long'], 1, -1)
    score_signal += np.where(df['rsi'] < rsi_threshold, 1, -1)
    score_signal += np.where(df['macd'] > df['macd_signal'], 1, -1)
    score_signal += df['ichimoku_signal']
    df['signal'] = np.where(score_signal > 1, 1, np.where(score_signal < -1, -1, 0))
    return df

# Fonction d’évaluation
def evaluate_strategy(individual, train_df):
    rsi_threshold, macd_threshold, stop_loss = individual
    ema_short, ema_long = optimize_ema(train_df)
    df = generate_signals(train_df.copy(), ema_short, ema_long, rsi_threshold, macd_threshold)
    final_capital = backtest_strategy(df, ema_short, ema_long, rsi_threshold, macd_threshold, stop_loss)
    individual.optimal_params = {
        'ema_short': ema_short,
        'ema_long': ema_long,
        'rsi_threshold': rsi_threshold,
        'macd_threshold': macd_threshold,
        'stop_loss': stop_loss,
        'fitness': final_capital
    }
    return final_capital,

# AG principal
if "FitnessMax" not in creator.__dict__:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_individual():
    return [random.randint(50, 80), random.uniform(0.0, 0.1), random.uniform(0.005, 0.05)]

toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.01, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate_strategy, train_df=train_df)

population = toolbox.population(n=10)
for generation in range(30):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < 0.7:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    
    for mutant in offspring:
        if random.random() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = list(map(toolbox.evaluate, invalid_ind))
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    population[:] = offspring
    
    best_individual = tools.selBest(population, 1)[0]
    print(f"Generation {generation}")
    print(f"Best Individual: RSI Threshold={best_individual[0]}, MACD Threshold={best_individual[1]:.4f}, Stop Loss={best_individual[2]:.4f}")
    print(f"Fitness: {best_individual.fitness.values[0]:.2f}")

# Résultats finaux et visualisation
best_individual = tools.selBest(population, 1)[0]
optimal_params = best_individual.optimal_params
train_df_with_signals = generate_signals(train_df.copy(), optimal_params['ema_short'], optimal_params['ema_long'],
                                         optimal_params['rsi_threshold'], optimal_params['macd_threshold'])
final_capital = backtest_strategy(train_df_with_signals, optimal_params['ema_short'], optimal_params['ema_long'],
                                  optimal_params['rsi_threshold'], optimal_params['macd_threshold'],
                                  optimal_params['stop_loss'])

print("\n=== Paramètres Optimaux ===")
print(f"EMA Short: {optimal_params['ema_short']}")
print(f"EMA Long: {optimal_params['ema_long']}")
print(f"RSI Threshold: {optimal_params['rsi_threshold']}")
print(f"MACD Threshold: {optimal_params['macd_threshold']:.4f}")
print(f"Stop Loss: {optimal_params['stop_loss']:.4f}")
print(f"Capital Final (Fitness avec frais): {optimal_params['fitness']:.2f}")



/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarn

KeyboardInterrupt: 

In [17]:
optimal_params

{'ema_short': 56.314272100098684,
 'ema_long': 113.57098913561161,
 'rsi_threshold': 46.027924061716966,
 'macd_threshold': 4.545650127138948,
 'fitness': np.float64(9875.279999999986)}

#### Basé uniquement sur Ichimoku

In [36]:
print(train_df)

                 timestamp     open     high      low    close      volume  \
0      2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614   
1      2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818   
2      2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272   
3      2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577   
4      2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796   
...                    ...      ...      ...      ...      ...         ...   
289255 2023-12-31 23:35:00  2279.86  2280.00  2277.66  2279.80   356.74170   
289256 2023-12-31 23:40:00  2279.80  2279.99  2279.25  2279.91   402.52030   
289257 2023-12-31 23:45:00  2279.91  2283.88  2279.64  2283.70   571.92240   
289258 2023-12-31 23:50:00  2283.71  2283.71  2280.47  2280.48   353.37290   
289259 2023-12-31 23:55:00  2280.48  2282.60  2279.79  2281.87   466.25540   

        open_btc  high_btc   low_btc  close_btc  ...  volatilit

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [42]:
import numpy as np
import pandas as pd

# Calcul des signaux Ichimoku sans futur
def calculate_ichimoku_signals(train_df):
    train_df = train_df.copy()
    # Assurer que l'index est en datetime
    if not pd.api.types.is_datetime64_any_dtype(train_df.index):
        train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
        train_df.set_index('timestamp', inplace=True)

    # Tenkan Sen (9 périodes)
    high_9 = train_df['high'].rolling(window=9, min_periods=1).max()
    low_9 = train_df['low'].rolling(window=9, min_periods=1).min()
    train_df['tenkan_sen'] = (high_9 + low_9) / 2

    # Kijun Sen (26 périodes)
    high_26 = train_df['high'].rolling(window=26, min_periods=1).max()
    low_26 = train_df['low'].rolling(window=26, min_periods=1).min()
    train_df['kijun_sen'] = (high_26 + low_26) / 2

    # Senkou Span A et B (calculés au présent, sans décalage futur)
    train_df['senkou_span_a'] = (train_df['tenkan_sen'] + train_df['kijun_sen']) / 2
    high_52 = train_df['high'].rolling(window=52, min_periods=1).max()
    low_52 = train_df['low'].rolling(window=52, min_periods=1).min()
    train_df['senkou_span_b'] = (high_52 + low_52) / 2

    # Kumo (Cloud)
    kumo_top = train_df[['senkou_span_a', 'senkou_span_b']].max(axis=1)
    kumo_bottom = train_df[['senkou_span_a', 'senkou_span_b']].min(axis=1)

    # Score des signaux
    score_signal = np.zeros(len(train_df))

    # 1. Tenkan Sen / Kijun Sen Cross
    tenkan_prev = train_df['tenkan_sen'].shift(1)
    kijun_prev = train_df['kijun_sen'].shift(1)
    bullish_cross = (train_df['tenkan_sen'] > train_df['kijun_sen']) & (tenkan_prev <= kijun_prev)
    bearish_cross = (train_df['tenkan_sen'] < train_df['kijun_sen']) & (tenkan_prev >= kijun_prev)
    score_signal += np.where(bullish_cross & (train_df['close'] > kumo_top), 1,  # Strong bullish
                            np.where(bullish_cross & (train_df['close'] < kumo_bottom), 0.25,  # Weak bullish
                                    np.where(bullish_cross, 0.5, 0)))  # Neutral bullish
    score_signal += np.where(bearish_cross & (train_df['close'] < kumo_bottom), -1,  # Strong bearish
                            np.where(bearish_cross & (train_df['close'] > kumo_top), -0.25,  # Weak bearish
                                    np.where(bearish_cross, -0.5, 0)))  # Neutral bearish

    # 2. Kijun Sen Cross
    price_prev = train_df['close'].shift(1)
    bullish_kijun = (train_df['close'] > train_df['kijun_sen']) & (price_prev <= train_df['kijun_sen'])
    bearish_kijun = (train_df['close'] < train_df['kijun_sen']) & (price_prev >= train_df['kijun_sen'])
    score_signal += np.where(bullish_kijun & (train_df['close'] > kumo_top), 1,
                            np.where(bullish_kijun & (train_df['close'] < kumo_bottom), 0.25,
                                    np.where(bullish_kijun, 0.5, 0)))
    score_signal += np.where(bearish_kijun & (train_df['close'] < kumo_bottom), -1,
                            np.where(bearish_kijun & (train_df['close'] > kumo_top), -0.25,
                                    np.where(bearish_kijun, -0.5, 0)))

    # 3. Kumo Breakout (sans futur)
    bullish_breakout = (train_df['close'] > kumo_top) & (price_prev <= kumo_top)
    bearish_breakout = (train_df['close'] < kumo_bottom) & (price_prev >= kumo_bottom)
    score_signal += np.where(bullish_breakout, 1, 0)
    score_signal += np.where(bearish_breakout, -1, 0)

    # Signal final
    train_df['ichimoku_signal'] = np.where(score_signal > 0.5, 1, np.where(score_signal < -0.5, -1, 0))
    return train_df


# Calculer les signaux Ichimoku
train_df_with_ichimoku = calculate_ichimoku_signals(train_df)

# Supprimer les limitations d'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Afficher train_df avec les signaux Ichimoku
print(train_df_with_ichimoku)

                        open     high      low    close      volume  open_btc  \
timestamp                                                                       
2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614  58739.46   
2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818  59071.84   
2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272  59030.00   
2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577  58965.01   
2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796  58883.57   
...                      ...      ...      ...      ...         ...       ...   
2023-12-31 23:35:00  2279.86  2280.00  2277.66  2279.80   356.74170  42229.26   
2023-12-31 23:40:00  2279.80  2279.99  2279.25  2279.91   402.52030  42231.10   
2023-12-31 23:45:00  2279.91  2283.88  2279.64  2283.70   571.92240  42241.09   
2023-12-31 23:50:00  2283.71  2283.71  2280.47  2280.48   353.37290  42269.82   
2023-12-31 23:55:00  2280.48

/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater_equal
  return op(a, b)
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [ ]:
import numpy as np
import pandas as pd
import random
from deap import base, creator, tools, algorithms

# Calcul de l'ATR
def calculate_atr(train_df_with_ichimoku, period=14):
    train_df_with_ichimoku = train_df_with_ichimoku.copy()
    high_low = train_df_with_ichimoku['high'] - train_df_with_ichimoku['low']
    high_close = np.abs(train_df_with_ichimoku['high'] - train_df_with_ichimoku['close'].shift(1))
    low_close = np.abs(train_df_with_ichimoku['low'] - train_df_with_ichimoku['close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    train_df_with_ichimoku['atr'] = true_range.rolling(window=period, min_periods=1).mean()
    return train_df_with_ichimoku

# Backtest avec stop loss basé sur ATR
def backtest_atr_strategy(train_df_with_ichimoku, atr_multiplier, initial_capital=5000, min_trade_delay=5):
    train_df_with_ichimoku = calculate_atr(train_df_with_ichimoku)
    signals = train_df_with_ichimoku['ichimoku_signal'].values
    close_prices = train_df_with_ichimoku['close'].values
    atr_values = train_df_with_ichimoku['atr'].values
    capital = initial_capital
    position = 0
    entry_price = 0
    last_trade_idx = -min_trade_delay - 1
    trade_count = 0

    for i in range(1, len(signals)):
        if signals[i] == 1 and position == 0 and capital >= close_prices[i] * 1.0005 and (i - last_trade_idx) > min_trade_delay:
            position = 1
            entry_price = close_prices[i]
            capital -= close_prices[i] * 1.0005  # Frais d'achat 0.05%
            last_trade_idx = i
            trade_count += 1
        elif position == 1:
            stop_loss_level = entry_price - (atr_values[i] * atr_multiplier)
            if close_prices[i] < stop_loss_level:  # Stop loss basé sur ATR
                capital += close_prices[i] * 0.9995
                position = 0
                last_trade_idx = i
                trade_count += 1
            elif signals[i] == -1 and (i - last_trade_idx) > min_trade_delay:
                capital += close_prices[i] * 0.9995  # Frais de vente
                position = 0
                last_trade_idx = i
                trade_count += 1
        current_value = capital + (position * close_prices[i])

    if position == 1:
        capital += close_prices[-1] * 0.9995
    penalty = trade_count * 5
    final_capital = capital - penalty
    return final_capital, trade_count

# AG pour optimiser le multiplicateur ATR
def optimize_atr(train_df_with_ichimoku):
    if "FitnessMax" not in creator.__dict__:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMax)

    def create_individual():
        return [random.uniform(1.0, 5.0)]  # Multiplicateur ATR entre 1 et 5

    def evaluate_individual(individual, train_df_with_ichimoku):
        atr_multiplier = individual[0]
        if atr_multiplier < 0:
            return 0,
        final_capital, _ = backtest_atr_strategy(train_df_with_ichimoku.copy(), atr_multiplier)
        return final_capital,

    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.5, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate_individual, train_df_with_ichimoku=train_df_with_ichimoku)

    population = toolbox.population(n=10)
    for generation in range(20):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                mutant[0] = max(1.0, min(5.0, mutant[0]))  # Limiter entre 1 et 5
                del mutant.fitness.values
        
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        population[:] = offspring

        best_individual = tools.selBest(population, 1)[0]
        print(f"Generation {generation}")
        print(f"Best ATR Multiplier: {best_individual[0]:.2f}")
        print(f"Fitness: {best_individual.fitness.values[0]:.2f}")

    best_individual = tools.selBest(population, 1)[0]
    return best_individual[0]

# Exécution de l'optimisation ATR sur train_df_with_ichimoku
# (Assumes train_df_with_ichimoku is already defined with ichimoku_signal)
best_atr_multiplier = optimize_atr(train_df_with_ichimoku)
final_capital, trade_count = backtest_atr_strategy(train_df_with_ichimoku.copy(), best_atr_multiplier)

# Résultats
print("\n=== Résultats Optimaux ===")
print(f"Best ATR Multiplier: {best_atr_multiplier:.2f}")
print(f"Capital Final (avec frais): {final_capital:.2f}")
print(f"Nombre de trades: {trade_count}")

In [38]:
print(train_df)

                 timestamp     open     high      low    close      volume  \
0      2021-04-01 00:00:00  1919.37  1937.82  1919.37  1934.49  5054.95614   
1      2021-04-01 00:05:00  1935.24  1935.25  1927.86  1927.90  1884.79818   
2      2021-04-01 00:10:00  1927.92  1929.63  1923.41  1926.55  2004.92272   
3      2021-04-01 00:15:00  1926.55  1928.76  1924.00  1924.80  1469.64577   
4      2021-04-01 00:20:00  1924.80  1929.78  1924.28  1928.29  1760.55796   
...                    ...      ...      ...      ...      ...         ...   
289255 2023-12-31 23:35:00  2279.86  2280.00  2277.66  2279.80   356.74170   
289256 2023-12-31 23:40:00  2279.80  2279.99  2279.25  2279.91   402.52030   
289257 2023-12-31 23:45:00  2279.91  2283.88  2279.64  2283.70   571.92240   
289258 2023-12-31 23:50:00  2283.71  2283.71  2280.47  2280.48   353.37290   
289259 2023-12-31 23:55:00  2280.48  2282.60  2279.79  2281.87   466.25540   

        open_btc  high_btc   low_btc  close_btc  ...  volatilit

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


### Modèle tout simple

In [15]:
import numpy as np
import pandas as pd
import random
from deap import base, creator, tools, algorithms

# Calcul de l'ATR
def calculate_atr(df, period=14):
    df = df.copy()
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift(1))
    low_close = np.abs(df['low'] - df['close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = true_range.rolling(window=period, min_periods=1).mean()
    return df

# Backtest avec EMA et stop loss ATR
def backtest_ema_atr_strategy(df, ema_short, ema_long, atr_multiplier, initial_capital=5000):
    df = calculate_atr(df)  # Calculer l’ATR
    df['ema_short'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    df['ema_long'] = df['close'].ewm(span=ema_long, adjust=False).mean()
    
    # Signal basé sur croisement EMA
    df['ema_atr_signal'] = np.where(df['ema_short'] > df['ema_long'], 1, -1)
    
    signals = df['ema_atr_signal'].values
    close_prices = df['close'].values
    atr_values = df['atr'].values
    capital = initial_capital
    position = 0
    entry_price = 0
    trade_count = 0

    for i in range(1, len(signals)):
        # Achat
        if signals[i] == 1 and position == 0 and capital >= close_prices[i] * 1.0005:
            position = 1
            entry_price = close_prices[i]
            capital -= close_prices[i] * 1.0005  # Frais 0.05%
            trade_count += 1
        # Vente sur signal ou stop loss ATR
        elif position == 1:
            stop_loss_level = entry_price - (atr_values[i] * atr_multiplier)
            if close_prices[i] < stop_loss_level:  # Stop loss déclenché
                capital += close_prices[i] * 0.9995
                position = 0
                trade_count += 1
            elif signals[i] == -1:
                capital += close_prices[i] * 0.9995  # Frais 0.05%
                position = 0
                trade_count += 1
    if position == 1:
        capital += close_prices[-1] * 0.9995  # Clôture
    penalty = trade_count * 5  # Pénalité par trade
    return capital - penalty

# AG pour optimiser EMA et ATR multiplier
def optimize_ema_atr(train_df):
    # Créer les classes DEAP (seulement si elles n’existent pas)
    if "FitnessMax" not in creator.__dict__:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMax)

    # Définition d’un individu : [ema_short, ema_long, atr_multiplier]
    def create_individual():
        return [
            random.randint(5, 50),        # EMA courte
            random.randint(50, 200),     # EMA longue
            random.uniform(1.0, 5.0)     # Multiplicateur ATR (1x à 5x)
        ]

    def evaluate_individual(individual, df):
        ema_short, ema_long, atr_multiplier = individual
        if ema_short < 1 or ema_long < 1 or ema_short >= ema_long or atr_multiplier < 0:
            return 0,
        return backtest_ema_atr_strategy(df, ema_short, ema_long, atr_multiplier),

    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate_individual, df=train_df)

    # Lancer l’AG
    population = toolbox.population(n=10)
    for generation in range(20):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        population[:] = offspring
        
        best_individual = tools.selBest(population, 1)[0]
        print(f"Generation {generation}")
        print(f"Best Individual: EMA Short={best_individual[0]}, EMA Long={best_individual[1]}, ATR Multiplier={best_individual[2]:.2f}")
        print(f"Fitness: {best_individual.fitness.values[0]:.2f}")

    # Résultats finaux
    best_individual = tools.selBest(population, 1)[0]
    print("\n=== Paramètres Optimaux ===")
    print(f"EMA Short: {best_individual[0]}")
    print(f"EMA Long: {best_individual[1]}")
    print(f"ATR Multiplier: {best_individual[2]:.2f}")
    print(f"Capital Final (Fitness avec frais): {best_individual.fitness.values[0]:.2f}")
    return best_individual

# Exemple d’utilisation
# Remplace train_df par ton DataFrame
# train_df = pd.read_csv("ton_fichier.csv")  # Si besoin
best_params = optimize_ema_atr(train_df)

Generation 0
Best Individual: EMA Short=47, EMA Long=162, ATR Multiplier=4.80
Fitness: -7030.91
Generation 1
Best Individual: EMA Short=47, EMA Long=156.86638903855928, ATR Multiplier=7.15
Fitness: -5334.93
Generation 2
Best Individual: EMA Short=47, EMA Long=156.86638903855928, ATR Multiplier=7.15
Fitness: -5334.93
Generation 3
Best Individual: EMA Short=47, EMA Long=162, ATR Multiplier=7.15
Fitness: -5094.50
Generation 4
Best Individual: EMA Short=47, EMA Long=162, ATR Multiplier=7.15
Fitness: -5094.50
Generation 5
Best Individual: EMA Short=47, EMA Long=162, ATR Multiplier=11.64
Fitness: -4657.63
Generation 6
Best Individual: EMA Short=48.4741056370762, EMA Long=162, ATR Multiplier=7.15
Fitness: -4523.60
Generation 7
Best Individual: EMA Short=48.4741056370762, EMA Long=162, ATR Multiplier=11.64
Fitness: -4221.56
Generation 8
Best Individual: EMA Short=48.4741056370762, EMA Long=162, ATR Multiplier=11.64
Fitness: -4221.56
Generation 9
Best Individual: EMA Short=48.4741056370762, EMA

backtest associées à la strat simple

In [23]:
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype
import random
from deap import base, creator, tools, algorithms
import matplotlib.pyplot as plt

# Calcul de l'ATR
def calculate_atr(df, period=14):
    df = df.copy()
    high_low = df['high'] - df['low']
    high_close = np.abs(df['high'] - df['close'].shift(1))
    low_close = np.abs(df['low'] - df['close'].shift(1))
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['atr'] = true_range.rolling(window=period, min_periods=1).mean()
    return df

# Backtest avec EMA et stop loss ATR
def backtest_ema_atr_strategy(df, ema_short, ema_long, atr_multiplier, initial_capital=5000):
    df = calculate_atr(df)
    df['ema_short'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    df['ema_long'] = df['close'].ewm(span=ema_long, adjust=False).mean()
    df['ema_atr_signal'] = np.where(df['ema_short'] > df['ema_long'], 1, -1)
    
    signals = df['ema_atr_signal'].values
    close_prices = df['close'].values
    atr_values = df['atr'].values
    capital = initial_capital
    position = 0
    entry_price = 0
    capital_history = [initial_capital]
    trade_count = 0

    for i in range(1, len(signals)):
        if signals[i] == 1 and position == 0 and capital >= close_prices[i] * 1.0005:
            position = 1
            entry_price = close_prices[i]
            capital -= close_prices[i] * 1.0005  # Frais 0.05%
            trade_count += 1
        elif position == 1:
            stop_loss_level = entry_price - (atr_values[i] * atr_multiplier)
            if close_prices[i] < stop_loss_level:  # Stop loss
                capital += close_prices[i] * 0.9995
                position = 0
                trade_count += 1
            elif signals[i] == -1:
                capital += close_prices[i] * 0.9995  # Frais 0.05%
                position = 0
                trade_count += 1
        # Mise à jour du capital (cash + position)
        current_value = capital + (position * close_prices[i])
        capital_history.append(current_value)
    
    if position == 1:
        capital += close_prices[-1] * 0.9995
    df['capital'] = pd.Series(capital_history + [capital] * (len(df) - len(capital_history)), index=df.index)
    return capital, trade_count

# AG pour optimiser EMA et ATR
def optimize_ema_atr(train_df):
    if "FitnessMax" not in creator.__dict__:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMax)

    def create_individual():
        return [random.randint(5, 50), random.randint(50, 200), random.uniform(1.0, 5.0)]

    def evaluate_individual(individual, df):
        ema_short, ema_long, atr_multiplier = individual
        if ema_short < 1 or ema_long < 1 or ema_short >= ema_long or atr_multiplier < 0:
            return 0,
        final_capital, _ = backtest_ema_atr_strategy(df, ema_short, ema_long, atr_multiplier)
        return final_capital,

    toolbox = base.Toolbox()
    toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("evaluate", evaluate_individual, df=train_df)

    population = toolbox.population(n=10)
    for generation in range(20):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.2:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        population[:] = offspring

    best_individual = tools.selBest(population, 1)[0]
    return {'ema_short': best_individual[0], 'ema_long': best_individual[1], 'atr_multiplier': best_individual[2]}

# Backtest sur plusieurs périodes
def multi_period_backtest(full_df):
    if not is_datetime64_any_dtype(full_df.index):
        full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])
        full_df.set_index('timestamp', inplace=True)

    # Périodes (2 ans train, 1 an test)
    periods = [
        ('202104_202303', '2021-04-01', '2023-03-31', '2023-04-01', '2024-03-31'),
        ('202101_202212', '2021-01-01', '2022-12-31', '2023-01-01', '2023-12-31'),
        ('202007_202206', '2020-07-01', '2022-06-30', '2022-07-01', '2023-06-30'),
        ('202010_202209', '2020-10-01', '2022-09-30', '2022-10-01', '2023-09-30'),
        ('202001_202112', '2020-01-01', '2021-12-31', '2022-01-01', '2022-12-31'),
    ]

    backtest_dfs = {}
    for period_name, train_start, train_end, test_start, test_end in periods:
        train_start = pd.to_datetime(train_start)
        train_end = pd.to_datetime(train_end)
        test_start = pd.to_datetime(test_start)
        test_end = pd.to_datetime(test_end)

        train_df = full_df[(full_df.index >= train_start) & (full_df.index <= train_end)].copy()
        test_df = full_df[(full_df.index >= test_start) & (full_df.index <= test_end)].copy()

        if len(train_df) == 0 or len(test_df) == 0:
            print(f"Période {period_name} : Données insuffisantes")
            continue

        # Optimisation sur train_df
        optimal_params = optimize_ema_atr(train_df)

        # Backtest sur test_df
        test_capital, test_trades = backtest_ema_atr_strategy(test_df, optimal_params['ema_short'],
                                                              optimal_params['ema_long'],
                                                              optimal_params['atr_multiplier'])
        


        # Stocker les résultats
        df_name = f"df_backtest_Periode_{period_name}"
        backtest_dfs[df_name] = pd.DataFrame({
            'train_start': [train_start],
            'train_end': [train_end],
            'test_start': [test_start],
            'test_end': [test_end],
            'ema_short': [optimal_params['ema_short']],
            'ema_long': [optimal_params['ema_long']],
            'atr_multiplier': [optimal_params['atr_multiplier']],
            'test_capital': [test_capital],
            'test_trades': [test_trades]
        })

        print(f"\n=== Résultats pour {df_name} ===")
        print(f"Train Period: {train_start} - {train_end}")
        print(f"Test Period: {test_start} - {test_end}")
        print(f"EMA Short: {optimal_params['ema_short']}")
        print(f"EMA Long: {optimal_params['ema_long']}")
        print(f"ATR Multiplier: {optimal_params['atr_multiplier']:.2f}")
        print(f"Test Capital: {test_capital:.2f}")
        print(f"Nombre de trades: {test_trades}")

    return backtest_dfs

# Exécuter le backtest
full_df = train_df  # Remplace par ton DataFrame complet
backtest_results = multi_period_backtest(full_df)

# Afficher les résultats
for df_name, df in backtest_results.items():
    print(f"\n{df_name}:")
    print(df)


=== Résultats pour df_backtest_Periode_202104_202303 ===
Train Period: 2021-04-01 00:00:00 - 2023-03-31 00:00:00
Test Period: 2023-04-01 00:00:00 - 2024-03-31 00:00:00
EMA Short: 44
EMA Long: 183.3536654710984
ATR Multiplier: 6.60
Test Capital: 4565.48
Nombre de trades: 694

=== Résultats pour df_backtest_Periode_202101_202212 ===
Train Period: 2021-01-01 00:00:00 - 2022-12-31 00:00:00
Test Period: 2023-01-01 00:00:00 - 2023-12-31 00:00:00
EMA Short: 45.36548414428389
EMA Long: 181.41860833809432
ATR Multiplier: 11.13
Test Capital: 4847.51
Nombre de trades: 822

=== Résultats pour df_backtest_Periode_202007_202206 ===
Train Period: 2020-07-01 00:00:00 - 2022-06-30 00:00:00
Test Period: 2022-07-01 00:00:00 - 2023-06-30 00:00:00
EMA Short: 42.286440056022506
EMA Long: 189.26083924058318
ATR Multiplier: 10.83
Test Capital: 5164.51
Nombre de trades: 738

=== Résultats pour df_backtest_Periode_202010_202209 ===
Train Period: 2020-10-01 00:00:00 - 2022-09-30 00:00:00
Test Period: 2022-10-01

In [18]:
import numpy as np
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype
import random
from deap import base, creator, tools, algorithms

# Backtest vectorisé
def backtest_strategy(df, ema_short, ema_long, rsi_threshold, macd_threshold, initial_capital=5000):
    signals = df['signal'].values
    close_prices = df['close'].values
    capital = initial_capital
    position = 0
    entry_price = 0

    for i in range(len(signals)):
        if signals[i] == 1 and position == 0 and capital >= close_prices[i]:
            position = 1
            entry_price = close_prices[i]
            capital -= close_prices[i]
        elif signals[i] == -1 and position == 1:
            capital += close_prices[i]
            position = 0
    if position == 1:
        capital += close_prices[-1]
    return capital

# Sous-AG pour optimiser les EMA
def optimize_ema(train_df):
    if "FitnessMax" not in creator.__dict__:
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    if "Individual" not in creator.__dict__:
        creator.create("Individual", list, fitness=creator.FitnessMax)

    def create_ema_individual():
        return [random.randint(5, 50), random.randint(50, 200)]

    def evaluate_ema(individual, df):
        ema_short, ema_long = individual
        if ema_short < 1 or ema_long < 1 or ema_short >= ema_long:
            return 0,
        ema_short_series = df['close'].ewm(span=ema_short, adjust=False).mean()
        ema_long_series = df['close'].ewm(span=ema_long, adjust=False).mean()
        df['signal'] = np.where(ema_short_series > ema_long_series, 1, -1)
        return backtest_strategy(df, ema_short, 0, 0, 0),

    ema_toolbox = base.Toolbox()
    ema_toolbox.register("individual", tools.initIterate, creator.Individual, create_ema_individual)
    ema_toolbox.register("population", tools.initRepeat, list, ema_toolbox.individual)
    ema_toolbox.register("mate", tools.cxTwoPoint)
    ema_toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
    ema_toolbox.register("select", tools.selTournament, tournsize=3)
    ema_toolbox.register("evaluate", evaluate_ema, df=train_df)

    population = ema_toolbox.population(n=5)
    for gen in range(10):
        offspring = ema_toolbox.select(population, len(population))
        offspring = list(map(ema_toolbox.clone, offspring))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < 0.7:
                ema_toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        for mutant in offspring:
            if random.random() < 0.2:
                ema_toolbox.mutate(mutant)
                del mutant.fitness.values
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = list(map(ema_toolbox.evaluate, invalid_ind))
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        population[:] = offspring

    best_ema = tools.selBest(population, 1)[0]
    return best_ema[0], best_ema[1]

# Générer les signaux avec paramètres optimaux
def generate_signals(df, ema_short, ema_long, rsi_threshold, macd_threshold):
    df['ema_short'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    df['ema_long'] = df['close'].ewm(span=ema_long, adjust=False).mean()
    df['eth_btc_ratio'] = df['close'] / df['close_btc']
    df['ratio_sma'] = df['eth_btc_ratio'].rolling(window=20, min_periods=1).mean()
    df['ratio_std'] = df['eth_btc_ratio'].rolling(window=20, min_periods=1).std()
    df['upper_band'] = df['ratio_sma'] + 2 * df['ratio_std']
    df['lower_band'] = df['ratio_sma'] - 2 * df['ratio_std']

    score_signal = np.zeros(len(df))
    score_signal += np.where(df['ema_short'] > df['ema_long'], 1, -1)
    score_signal += np.where(df['rsi'] < rsi_threshold, 1, -1)
    score_signal += np.where(df['macd'] > df['macd_signal'], 1, -1)
    df['signal'] = np.where(score_signal > 0, 1, -1)

# Fonction d’évaluation optimisée
def evaluate_strategy(individual, train_df):
    rsi_threshold, macd_threshold = individual
    ema_short, ema_long = optimize_ema(train_df)
    generate_signals(train_df, ema_short, ema_long, rsi_threshold, macd_threshold)
    final_capital = backtest_strategy(train_df, ema_short, ema_long, rsi_threshold, macd_threshold)
    individual.optimal_params = {
        'ema_short': ema_short,
        'ema_long': ema_long,
        'rsi_threshold': rsi_threshold,
        'macd_threshold': macd_threshold,
        'fitness': final_capital
    }
    return final_capital,

# Backtest sur plusieurs périodes
def multi_period_backtest(full_df):
    # Assurer que l’index est en datetime
    if not is_datetime64_any_dtype(full_df.index):
        full_df['timestamp'] = pd.to_datetime(full_df['timestamp'])
        full_df.set_index('timestamp', inplace=True)

    # Définir les périodes (2 ans train, 1 an test, k=5)
    periods = [
        ('202104_202303', '2021-04-01', '2023-03-31', '2023-04-01', '2024-03-31'),  # Train: 2 ans, Test: 1 an (limité à 2024-01-01 ici)
        ('202101_202212', '2021-01-01', '2022-12-31', '2023-01-01', '2023-12-31'),  # Si données avant avril 2021
        ('202007_202206', '2020-07-01', '2022-06-30', '2022-07-01', '2023-06-30'),  # Si données disponibles
        ('202010_202209', '2020-10-01', '2022-09-30', '2022-10-01', '2023-09-30'),  # Si données disponibles
        ('202001_202112', '2020-01-01', '2021-12-31', '2022-01-01', '2022-12-31'),  # Si données disponibles
    ]

    # Dictionnaire pour stocker les DataFrames
    backtest_dfs = {}

    # AG principal pour chaque période
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()

    def create_individual():
        return [random.randint(50, 80), random.uniform(0.0, 0.1)]

    toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=5, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    for period_name, train_start, train_end, test_start, test_end in periods:
        train_start = pd.to_datetime(train_start)
        train_end = pd.to_datetime(train_end)
        test_start = pd.to_datetime(test_start)
        test_end = pd.to_datetime(test_end)

        # Filtrer les périodes
        train_df = full_df[(full_df.index >= train_start) & (full_df.index <= train_end)].copy()
        test_df = full_df[(full_df.index >= test_start) & (full_df.index <= test_end)].copy()

        if len(train_df) == 0 or len(test_df) == 0:
            print(f"Période {period_name} : Données insuffisantes")
            continue

        # Entraîner sur train_df
        toolbox.register("evaluate", evaluate_strategy, train_df=train_df)
        population = toolbox.population(n=5)
        for generation in range(20):
            offspring = toolbox.select(population, len(population))
            offspring = list(map(toolbox.clone, offspring))
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < 0.7:
                    toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values
            for mutant in offspring:
                if random.random() < 0.2:
                    toolbox.mutate(mutant)
                    del mutant.fitness.values
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = list(map(toolbox.evaluate, invalid_ind))
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = fit
            population[:] = offspring

        best_individual = tools.selBest(population, 1)[0]
        optimal_params = best_individual.optimal_params

        # Tester sur test_df avec paramètres optimaux
        generate_signals(test_df, optimal_params['ema_short'], optimal_params['ema_long'],
                        optimal_params['rsi_threshold'], optimal_params['macd_threshold'])
        test_capital = backtest_strategy(test_df, optimal_params['ema_short'], optimal_params['ema_long'],
                                        optimal_params['rsi_threshold'], optimal_params['macd_threshold'])

        # Créer DataFrame de résultats
        df_name = f"df_backtest_Periode_{period_name}"
        backtest_dfs[df_name] = pd.DataFrame({
            'train_start': [train_start],
            'train_end': [train_end],
            'test_start': [test_start],
            'test_end': [test_end],
            'ema_short': [optimal_params['ema_short']],
            'ema_long': [optimal_params['ema_long']],
            'rsi_threshold': [optimal_params['rsi_threshold']],
            'macd_threshold': [optimal_params['macd_threshold']],
            'train_capital': [optimal_params['fitness']],
            'test_capital': [test_capital]
        })

        print(f"\n=== Résultats pour {df_name} ===")
        print(f"Train Period: {train_start} - {train_end}")
        print(f"Test Period: {test_start} - {test_end}")
        print(f"EMA Short: {optimal_params['ema_short']}")
        print(f"EMA Long: {optimal_params['ema_long']}")
        print(f"RSI Threshold: {optimal_params['rsi_threshold']}")
        print(f"MACD Threshold: {optimal_params['macd_threshold']:.4f}")
        print(f"Train Capital: {optimal_params['fitness']:.2f}")
        print(f"Test Capital: {test_capital:.2f}")

    return backtest_dfs

# Exécuter le backtest
full_df = train_df  # Remplace par ton DataFrame complet
backtest_results = multi_period_backtest(full_df)

# Accéder aux DataFrames (exemple)
for df_name, df in backtest_results.items():
    print(f"\n{df_name}:")
    print(df)

c:\Users\Liam\AppData\Local\Programs\Python\Python313\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Liam\AppData\Local\Programs\Python\Python313\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "



=== Résultats pour df_backtest_Periode_202104_202303 ===
Train Period: 2021-04-01 00:00:00 - 2023-03-31 00:00:00
Test Period: 2023-04-01 00:00:00 - 2024-03-31 00:00:00
EMA Short: 40
EMA Long: 121.41976874154271
RSI Threshold: 66.98033698626637
MACD Threshold: 0.0611
Train Capital: 7150.24
Test Capital: 5256.40

=== Résultats pour df_backtest_Periode_202101_202212 ===
Train Period: 2021-01-01 00:00:00 - 2022-12-31 00:00:00
Test Period: 2023-01-01 00:00:00 - 2023-12-31 00:00:00
EMA Short: 47
EMA Long: 167
RSI Threshold: 35.74212069286609
MACD Threshold: 11.1715
Train Capital: 9979.37
Test Capital: 5487.25

=== Résultats pour df_backtest_Periode_202007_202206 ===
Train Period: 2020-07-01 00:00:00 - 2022-06-30 00:00:00
Test Period: 2022-07-01 00:00:00 - 2023-06-30 00:00:00
EMA Short: 42.99173000591738
EMA Long: 118
RSI Threshold: 63
MACD Threshold: 0.0530
Train Capital: 6413.79
Test Capital: 5680.33

=== Résultats pour df_backtest_Periode_202010_202209 ===
Train Period: 2020-10-01 00:00:0

In [19]:
train_df.to_csv('train_df_without_ichimoku.csv', index=True)

J'applique les valeurs aux test

{'ema_short': 56.314272100098684,
 'ema_long': 113.57098913561161,
 'rsi_threshold': 46.027924061716966,
 'macd_threshold': 4.545650127138948,

In [22]:
import numpy as np
import pandas as pd

# Fonction pour générer les signaux avec tes paramètres optimaux
def apply_optimal_signals(df, ema_short=56, ema_long=113, rsi_threshold=46, macd_threshold=4.545650127138948):
    # S’assurer que l’index est en datetime
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df.set_index('timestamp', inplace=True)

    # Calculs des EMA avec les paramètres optimaux
    df['ema_short'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    df['ema_long'] = df['close'].ewm(span=ema_long, adjust=False).mean()

    # Calcul du ratio ETH/BTC et Bollinger Bands
    df['eth_btc_ratio'] = df['close'] / df['close_btc']
    df['ratio_sma'] = df['eth_btc_ratio'].rolling(window=20, min_periods=1).mean()
    df['ratio_std'] = df['eth_btc_ratio'].rolling(window=20, min_periods=1).std()
    df['upper_band'] = df['ratio_sma'] + 2 * df['ratio_std']
    df['lower_band'] = df['ratio_sma'] - 2 * df['ratio_std']

    # Score signal basé sur tes indicateurs et seuils optimaux
    score_signal = np.zeros(len(df))
    score_signal += np.where(df['ema_short'] > df['ema_long'], 1, -1)  # EMA croisement
    score_signal += np.where(df['rsi'] < rsi_threshold, 1, -1)  # RSI seuil
    score_signal += np.where(df['macd'] > df['macd_signal'], 1, -1)  # MACD vs signal (MACD Threshold non utilisé ici car déjà dans les données)


    # Signal binaire : 1 (achat), -1 (vente)
    df['signal'] = np.where(score_signal > 0, 1, -1)

# Appliquer sur ton test_df
apply_optimal_signals(test_df)

# Afficher un aperçu
print("Aperçu de test_df avec les signaux :")
print(test_df[['close', 'close_btc', 'ema_short', 'ema_long', 'rsi', 'macd', 'macd_signal', 'volatility_20d', 'volatility_20d_bnb', 'signal']].head(10))

# Vérifier les colonnes
print("\nColonnes disponibles dans test_df après calcul :")
print(test_df.columns)

# Exporter en CSV si tu veux
test_df.to_csv('C:/Users/Liam/Desktop/Projet crypto/Optimisation/test_df_with_signals.csv', index=True)
print("Export terminé : test_df_with_signals.csv créé !")

Aperçu de test_df avec les signaux :
                       close  close_btc    ema_short     ema_long  rsi  \
timestamp                                                                
2024-01-01 00:00:00  2287.84   42397.23  2287.840000  2287.840000  NaN   
2024-01-01 00:05:00  2290.77   42409.96  2287.942807  2287.891404  NaN   
2024-01-01 00:10:00  2296.71   42488.00  2288.250428  2288.046116  NaN   
2024-01-01 00:15:00  2292.36   42458.85  2288.394623  2288.121798  NaN   
2024-01-01 00:20:00  2294.67   42473.93  2288.614812  2288.236679  NaN   
2024-01-01 00:25:00  2291.05   42419.73  2288.700257  2288.286035  NaN   
2024-01-01 00:30:00  2289.78   42362.93  2288.738143  2288.312245  NaN   
2024-01-01 00:35:00  2293.95   42447.82  2288.921015  2288.411153  NaN   
2024-01-01 00:40:00  2293.22   42441.32  2289.071857  2288.495519  NaN   
2024-01-01 00:45:00  2292.24   42422.46  2289.183020  2288.561211  NaN   

                         macd  macd_signal  volatility_20d  \
timestamp   

Signaux Ichimoku

Aperçu de train_df avec signaux Ichimoku (sans futur) :
                       close     high      low  tenkan_sen  kijun_sen  \
timestamp                                                               
2021-04-01 00:00:00  1934.49  1937.82  1919.37    1928.595   1928.595   
2021-04-01 00:05:00  1927.90  1935.25  1927.86    1928.595   1928.595   
2021-04-01 00:10:00  1926.55  1929.63  1923.41    1928.595   1928.595   
2021-04-01 00:15:00  1924.80  1928.76  1924.00    1928.595   1928.595   
2021-04-01 00:20:00  1928.29  1929.78  1924.28    1928.595   1928.595   
2021-04-01 00:25:00  1924.24  1929.84  1923.81    1928.595   1928.595   
2021-04-01 00:30:00  1928.56  1929.26  1924.23    1928.595   1928.595   
2021-04-01 00:35:00  1922.49  1930.00  1920.91    1928.595   1928.595   
2021-04-01 00:40:00  1923.51  1925.67  1921.34    1928.595   1928.595   
2021-04-01 00:45:00  1926.00  1928.18  1923.00    1928.080   1928.595   

                     senkou_span_a_present  senkou_span_b_present  

# 